# TensorFlow Dataset API

En este ejemplo, mostraremos cómo cargar datos de arreglos numéricos en el nuevo archivo 
TensorFlow 'Dataset' API. El Dataset API implementa un canal de datos optimizado
con colas, que agilizan el procesamiento de datos y la formación (especialmente en la GPU).

In [1]:
import tensorflow as tf

# Importar datos MNIST (Numpy format)
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parametros
learning_rate = 0.01
num_steps = 1000
batch_size = 128
display_step = 100

# Network Parametros
n_input = 784 # Entrada de datos MNIST (forma de la imagen: 28*28)
n_classes = 10 # Clases totales del MNIST (0-9 dígitos)
dropout = 0.75 # Abandono, probabilidad de mantener las unidades

sess = tf.Session()

# Crear un tensor de conjunto de datos a partir de las imágenes y las etiquetas
dataset = tf.contrib.data.Dataset.from_tensor_slices(
    (mnist.train.images, mnist.train.labels))
# Crear lotes de datos
dataset = dataset.batch(batch_size)
# Crear un iterador, para repasar el conjunto de datos
iterator = dataset.make_initializable_iterator()
# Es mejor usar 2 marcadores de posición, para evitar cargar todos los datos en la memoria,
# y evitar la longitud de restricción de 2Gb de un tensor.
_data = tf.placeholder(tf.float32, [None, n_input])
_labels = tf.placeholder(tf.float32, [None, n_classes])
# Inicializar el iterador
sess.run(iterator.initializer, feed_dict={_data: mnist.train.images,
                                          _labels: mnist.train.labels})

# Entrada de red neuronal
X, Y = iterator.get_next()

In [3]:
# -----------------------------------------------
# ESTA ES UNA CNN CLÁSICA (ver ejemplos, sección 3)
# -----------------------------------------------
# Tenga en cuenta que algunos elementos han cambiado (uso de sess run).

# Crear modelo
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Definir un ámbito de reutilización de las variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # La entrada de datos MNIST es un vector 1-D de 784 características (28*28 píxeles)
        # Modificar para que coincida con el formato de la imagen[Altura x Anchura x Canal]
        # La entrada del tensor se convierte en 4-D:[Tamaño del lote, altura, ancho, canal]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Capa de convolución con 32 filtros y un tamaño de kernel de 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Agrupación máxima (muestreo descendente) con zancadas de 2 y tamaño de kernel de 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Capa de convolución con 32 filtros y un tamaño de kernel de 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Agrupación máxima (muestreo descendente) con zancadas de 2 y tamaño de kernel de 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Aplanar los datos a un vector 1-D para la capa completamente conectada.
        fc1 = tf.contrib.layers.flatten(conv2)

        # Capa completamente conectada (en la carpeta contrib por ahora)
        fc1 = tf.layers.dense(fc1, 1024)
        # Aplicar Abandono (si is_training es Falso, no se aplica el abandono)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Capa de salida, predicción de clases
        out = tf.layers.dense(fc1, n_classes)
        # Porque 'softmax_cross_entropy_with_logits' ya aplica softmax,
        # sólo aplicamos softmax a la red de pruebas
        out = tf.nn.softmax(out) if not is_training else out

    return out


# Debido a que los Abandonados tienen un comportamiento diferente en el entrenamiento y en el tiempo de predicción, nosotros
# necesitan crear 2 gráficos de cálculo distintos que compartan los mismos pesos.

# Cree un gráfico para el entrenamiento
logits_train = conv_net(X, n_classes, dropout, reuse=False, is_training=True)
# Cree otro gráfico para probar que reutilice los mismos pesos, pero que tenga
# comportamiento diferente para la "deserción escolar" (no se aplica).
logits_test = conv_net(X, n_classes, dropout, reuse=True, is_training=False)

# Definir la pérdida y el optimizador (con los registros del tren, para que la pérdida de datos tenga efecto)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluar el modelo (con registros de pruebas, para deshabilitar la deserción escolar)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [4]:
# Inicializar las variables (es decir, asignar su valor por defecto)
init = tf.global_variables_initializer()

# Ejecutar el inicializador
sess.run(init)

# Circulo de entrenamiento
for step in range(1, num_steps + 1):
    
    try:
        # Ejecutar Optimización
        sess.run(train_op)
    except tf.errors.OutOfRangeError:
        # Recargar el iterador cuando llegue al final del conjunto de datos
        sess.run(iterator.initializer, 
                 feed_dict={_data: mnist.train.images,
                            _labels: mnist.train.labels})
        sess.run(train_op)
        
    if step % display_step == 0 or step == 1:
        # Calcular la pérdida y la precisión de los lotes
        # (note que esto consume un nuevo lote de datos)
        loss, acc = sess.run([loss_op, accuracy])
        print("Step " + str(step) + ", Minibatch Loss= " + \
              "{:.4f}".format(loss) + ", Training Accuracy= " + \
              "{:.3f}".format(acc))

print("Optimization Finished!")

Step 1, Minibatch Loss= 7.9429, Training Accuracy= 0.070
Step 100, Minibatch Loss= 0.3491, Training Accuracy= 0.922
Step 200, Minibatch Loss= 0.2343, Training Accuracy= 0.922
Step 300, Minibatch Loss= 0.1838, Training Accuracy= 0.969
Step 400, Minibatch Loss= 0.1715, Training Accuracy= 0.953
Step 500, Minibatch Loss= 0.2730, Training Accuracy= 0.938
Step 600, Minibatch Loss= 0.3427, Training Accuracy= 0.953
Step 700, Minibatch Loss= 0.2261, Training Accuracy= 0.961
Step 800, Minibatch Loss= 0.1487, Training Accuracy= 0.953
Step 900, Minibatch Loss= 0.1438, Training Accuracy= 0.945
Step 1000, Minibatch Loss= 0.1786, Training Accuracy= 0.961
Optimization Finished!
